In [55]:
import pandas as pd
import geopandas as gpd
import numpy as np
import scipy as sp
from shapely.geometry import Point
from geopandas import GeoDataFrame

In [56]:
raw_airbnb=pd.read_csv("data/listings.csv")
london_pois=gpd.read_file("data/pois/gis_osm_pois_free_1.shp")
msoa_raw_boundary=gpd.read_file("data/greater_london.gpkg")

In [57]:
geometry = [Point(xy) for xy in zip(raw_airbnb['longitude'], raw_airbnb['latitude'])]
geo_airbnb = gpd.GeoDataFrame(raw_airbnb, geometry=geometry, crs="EPSG:4326")
# 导入爱彼迎CSV，转换为GeoDataFrame

In [58]:
pois_27700 = london_pois.to_crs("EPSG:27700")
geo_airbnb=geo_airbnb.to_crs("EPSG:27700")

london_pois = GeoDataFrame(london_pois, geometry=london_pois['geometry'])
geo_airbnb = GeoDataFrame(geo_airbnb, geometry=geo_airbnb['geometry'])
#重投影统一坐标系

In [85]:
#筛掉不需要的列
geo_airbnb=geo_airbnb.drop(columns=['id', 'name', 'host_id', 'host_name', 'neighbourhood_group', 'calculated_host_listings_count','availability_365'])

KeyError: "['id', 'name', 'host_id', 'host_name', 'neighbourhood_group', 'calculated_host_listings_count', 'availability_365'] not found in axis"

In [65]:
airbnb_intersect = gpd.sjoin(geo_airbnb, msoa_raw_boundary, how="inner", predicate="intersects")
#交汇爱彼迎数据与空间

In [66]:
airbnb_intersect

,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,number_of_reviews_ltm,license,geometry,index_right,geo_code,geo_label,geo_labelw
0,Enfield,51.611370,-0.118880,Private room,40.0,7,38,2024-01-01,0.28,1,NaN,POINT (530350.368 191957.865),978,E02000312,Enfield 036,None
1,Hammersmith and Fulham,51.501550,-0.233002,Entire home/apt,131.0,5,91,2024-06-09,0.59,10,NaN,POINT (522743.872 179548.046),899,E02000376,Hammersmith and Fulham 005,None
2,Camden,51.559528,-0.144319,Entire home/apt,215.0,5,216,2024-07-09,1.35,5,NaN,POINT (528735.364 186147.794),836,E02000168,Camden 003,None
3,Richmond upon Thames,51.447770,-0.339620,Private room,NaN,2,0,NaN,NaN,0,NaN,POINT (515479.62 173394.164),338,E02000797,Richmond upon Thames 014,None
4,Lambeth,51.491476,-0.111514,Entire home/apt,150.0,30,148,2024-06-15,0.93,12,NaN,POINT (531204.63 178638.342),829,E02000619,Lambeth 002,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96177,Camden,51.547717,-0.143016,Entire home/apt,155.0,1,0,NaN,NaN,0,NaN,POINT (528859.074 184836.65),847,E02000172,Camden 007,None
96178,Southwark,51.505380,-0.091734,Entire home/apt,238.0,5,0,NaN,NaN,0,NaN,POINT (532537.407 180220.082),118,E02000808,Southwark 002,None
96179,Southwark,51.505380,-0.091734,Entire home/apt,188.0,5,0,NaN,NaN,0,NaN,POINT (532537.407 180220.082),118,E02000808,Southwark 002,None
96180,Southwark,51.505380,-0.091734,Entire home/apt,457.0,5,0,NaN,NaN,0,NaN,POINT (532537.407 180220.082),118,E02000808,Southwark 002,None


In [78]:
categories = london_pois['fclass'].unique()
categories
#查看poi类型

array(['waste_basket', 'bicycle_rental', 'tourist_info', 'pub',
       'restaurant', 'post_box', 'camera_surveillance', 'library',
       'post_office', 'telephone', 'fountain', 'artwork', 'toilet',
       'cafe', 'pitch', 'recycling', 'bar', 'cinema', 'memorial',
       'monument', 'recycling_glass', 'convenience', 'furniture_shop',
       'laundry', 'courthouse', 'school', 'sports_centre', 'attraction',
       'supermarket', 'university', 'beverages', 'fast_food',
       'fire_station', 'water_tower', 'newsagent', 'doctors',
       'department_store', 'museum', 'hotel', 'playground', 'gift_shop',
       'atm', 'veterinary', 'pharmacy', 'dentist', 'chemist', 'theatre',
       'bank', 'bicycle_shop', 'car_rental', 'bakery', 'hairdresser',
       'doityourself', 'comms_tower', 'hospital', 'car_sharing',
       'archaeological', 'town_hall', 'nursing_home', 'embassy',
       'recycling_paper', 'college', 'viewpoint', 'kindergarten',
       'recycling_clothes', 'public_building', 'arts_ce

In [80]:
filtered_pois = pois_27700[pois_27700['fclass'].isin(['hotel', 'guesthouse', 'motel', 'hostel', 'chalet'])]
#从osm过滤与住宿酒店有关的poi

In [87]:
#交汇poi和msoa
pois_intersect = gpd.sjoin(filtered_pois,msoa_raw_boundary, how="inner", predicate="intersects")

In [88]:
pois_intersect

,osm_id,code,fclass,name,geometry,index_right,geo_code,geo_label,geo_labelw
216,29372721,2401,hotel,Sundridge Park hotel and conference Ctr,POINT (541760.234 170636.789),783,E02006782,Bromley 040,None
315,31314241,2401,hotel,DoubleTree by Hilton Hotel London - Marble Arch,POINT (528007.431 181126.739),531,E02000971,Westminster 012,None
674,59931333,2401,hotel,easyHotel Victoria,POINT (529086.504 178641.128),331,E02000981,Westminster 022,None
1331,254025974,2401,hotel,Crestfield Hotel,POINT (530338.674 182875.701),341,E02000189,Camden 024,None
1332,254025975,2401,hotel,Kings Cross Inn Hotel,POINT (530301.093 182931.054),341,E02000189,Camden 024,None
...,...,...,...,...,...,...,...,...,...
89067,12139667177,2401,hotel,Spark by Hilton London Romford,POINT (551364.683 188537.227),27,E02000476,Havering 013,None
89245,12152771492,2401,hotel,Hotel Xanadu,POINT (517653.657 180568.267),363,E02000257,Ealing 020,None
89402,12167570871,2401,hotel,Travelodge,POINT (518239.169 169276.216),102,E02000606,Kingston upon Thames 009,None
89883,12199188775,2401,hotel,Diana Hotel,POINT (522143.571 183036.695),684,E02000123,Brent 031,None


In [97]:
accommodation_summerize=(
    pois_intersect
    .groupby(['geo_code','fclass'])
    .agg({
        'geo_label': 'first',
    })

    .reset_index()
)
#分类酒店类型和区域的列表

In [98]:
accommodation_summerize#分类酒店类型和区域的列表

,geo_code,fclass,geo_label
0,E02000001,guesthouse,City of London 001
1,E02000001,hostel,City of London 001
2,E02000001,hotel,City of London 001
3,E02000016,hotel,Barking and Dagenham 015
4,E02000031,hotel,Barnet 008
...,...,...,...
215,E02006802,hostel,Southwark 034
216,E02006854,hotel,Tower Hamlets 033
217,E02006928,hostel,Greenwich 035
218,E02006928,hotel,Greenwich 035


In [109]:
#汇总酒店数量并且重命名列
hotel_summarize = (
    accommodation_summerize
    .groupby(['geo_code'])
    .agg({
        'geo_label': 'first',  # 获取每个分组的 geo_label 第一条记录
        'fclass': 'count'      # 统计每组的 fclass 数量（如酒店数量）
    })
    .reset_index()
)

hotel_summarize.rename(columns={'fclass': 'count'}, inplace=True)

# 查看结果
print(hotel_summarize)

      geo_code                 geo_label  count
0    E02000001        City of London 001      3
1    E02000016  Barking and Dagenham 015      1
2    E02000031                Barnet 008      1
3    E02000051                Barnet 028      1
4    E02000057                Barnet 034      1
..         ...                       ...    ...
181  E02006801               Lambeth 036      2
182  E02006802             Southwark 034      1
183  E02006854         Tower Hamlets 033      1
184  E02006928             Greenwich 035      2
185  E02006930             Greenwich 037      1

[186 rows x 3 columns]


In [111]:
hotel_summarize

,geo_code,geo_label,count
0,E02000001,City of London 001,3
1,E02000016,Barking and Dagenham 015,1
2,E02000031,Barnet 008,1
3,E02000051,Barnet 028,1
4,E02000057,Barnet 034,1
...,...,...,...
181,E02006801,Lambeth 036,2
182,E02006802,Southwark 034,1
183,E02006854,Tower Hamlets 033,1
184,E02006928,Greenwich 035,2
